In [1]:
from copy import deepcopy

import numpy as np
import pandas as pd

import datetime

import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError

In [2]:
CONFIGS = {
    'data_path': '../data/',
    'model_path': '../model/',
    'model_name': 'multi_input',
    'model_type': 'cnn1d',
    
    'valid_start_index': 1704,
    'test_start_index': 1872,
    
    'batch_size': 64,
    'learning_rate': 1e-4,
    'epochs': 100,
    'es_patience': 10,
    
    'window_size': 7*24,
    'target_length': 3,
}

In [3]:
train_origin = pd.read_csv(CONFIGS['data_path']+'train.csv', encoding='cp949')

In [4]:
data = deepcopy(train_origin)

data.columns = [
    'num', 'date_time', 'target', 'temp', 'wind',
    'humid', 'rain', 'sun', 'non_elec_eq', 'sunlight_eq'
]

data = data.loc[data['num'] == 1, :]

print(f'data.shape: {data.shape}')

data.shape: (2040, 10)


In [5]:
def mk_time_data(data):
    
    new_data = data.copy()

    new_data['date_time'] = data['date_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H'))
    
    new_data['time_stamp'] = new_data['date_time'].apply(lambda x: x.timestamp())
    
    new_data['year'] = new_data['date_time'].apply(lambda x: x.year)
    new_data['month'] = new_data['date_time'].apply(lambda x: x.month)
    new_data['day'] = new_data['date_time'].apply(lambda x: x.day)
    
    new_data['hour'] = new_data['date_time'].apply(lambda x: x.hour)
    new_data['cos_hour'] = np.cos(2*np.pi*(new_data['hour']/24))
    new_data['sin_hour'] = np.sin(2*np.pi*(new_data['hour']/24))

    new_data['weekday'] = new_data['date_time'].apply(lambda x: x.weekday())
    new_data['cos_weekday'] = np.cos(2*np.pi*(new_data['weekday']/7))
    new_data['sin_weekday'] = np.sin(2*np.pi*(new_data['weekday']/7))
    
    new_data['is_holiday'] = 0
    new_data.loc[(new_data['weekday'] == 5) | (new_data['weekday'] == 6), 'is_holiday'] = 1
    new_data.loc[(new_data['month'] == 8) & (new_data['day'] == 17), 'is_holiday'] = 1
    
    return new_data

In [6]:
new_data = mk_time_data(data)

In [7]:
def mk_mean_std_dict(data):
    mean_std_dict = {
        col: {
            'mean': data[col].mean(),
            'std': data[col].std()
        } for col in data.columns
    }
    return mean_std_dict

In [8]:
scaling_cols = [
    'temp', 'wind', 'humid', 'rain', 'sun', 'time_stamp', 'target'
]

mean_std_dict = mk_mean_std_dict(new_data[scaling_cols][:CONFIGS['valid_start_index']])
CONFIGS['mean_std_dict'] = mean_std_dict

In [9]:
def standard_scaling(data, mean_std_dict=None):
    if not mean_std_dict:
        mean_std_dict = mk_mean_std_dict(data)
    new_data = data.copy()
    for col in new_data.columns:
        new_data[col] -= mean_std_dict[col]['mean']
        new_data[col] /= mean_std_dict[col]['std']
    return new_data

In [10]:
new_data[scaling_cols] = standard_scaling(new_data[scaling_cols])

In [11]:
time_series_cols = [
    'temp', 'wind', 'humid', 'rain', 'sun', 'time_stamp',
    'cos_hour', 'sin_hour', 'cos_weekday', 'sin_weekday',
    'is_holiday', 'target',
]
target_date_info_cols = [
    'temp', 'wind', 'humid', 'rain', 'sun', 'time_stamp',
    'cos_hour', 'sin_hour', 'cos_weekday', 'sin_weekday',
    'is_holiday',
]
target_cols = ['target']

CONFIGS['time_series_cols'] = time_series_cols
CONFIGS['target_date_info_cols'] = target_date_info_cols
CONFIGS['target_cols'] = target_cols

In [12]:
def mk_dataset(data, CONFIGS, shuffle=False):
    
    time_series = data[CONFIGS['time_series_cols']][:-CONFIGS['target_length']]
    target_date_info = data[CONFIGS['target_date_info_cols']][CONFIGS['window_size']+1:-(CONFIGS['target_length']-2)]
    target = data[CONFIGS['target_cols']][CONFIGS['window_size']:]
    
    time_series_ds = Dataset.from_tensor_slices(time_series)
    time_series_ds = time_series_ds.window(CONFIGS['window_size'], shift=1, drop_remainder=True)
    time_series_ds = time_series_ds.flat_map(lambda x: x).batch(CONFIGS['window_size'])
    
    target_date_info_ds = Dataset.from_tensor_slices(target_date_info)
    
    target_ds = Dataset.from_tensor_slices(target)
    target_ds = target_ds.window(CONFIGS['target_length'], shift=1, drop_remainder=True)
    target_ds = target_ds.flat_map(lambda x: x).batch(CONFIGS['target_length'])
    
    ds = Dataset.zip(((time_series_ds, target_date_info_ds), target_ds))
    if shuffle:
        ds = ds.shuffle(512)
    ds = ds.batch(CONFIGS['batch_size']).cache().prefetch(2)
    
    return ds

In [13]:
train = new_data.loc[:CONFIGS['valid_start_index'], :]
valid = new_data.loc[CONFIGS['valid_start_index']-CONFIGS['window_size']:CONFIGS['test_start_index'], :]
test = new_data.loc[CONFIGS['test_start_index']-CONFIGS['window_size']:, :]

train_ds = mk_dataset(train, CONFIGS, shuffle=True)
valid_ds = mk_dataset(valid, CONFIGS)
test_ds = mk_dataset(test, CONFIGS)

In [14]:
def inversed_rmse(y_true, y_pred, mean, std):
    y_true = (y_true+mean)*std
    y_pred = (y_pred+mean)*std
    mse = tf.reduce_mean((y_true-y_pred)**2)
    return tf.sqrt(mse)

inversed_rmse_metric = lambda y_true, y_pred: inversed_rmse(y_true, y_pred, **CONFIGS['mean_std_dict']['target'])

In [15]:
def set_model(CONFIGS, model_name=None, print_summary=False):
    
    time_series_inputs = Input(batch_shape=(
        None, CONFIGS['window_size'], len(CONFIGS['time_series_cols'])
    ), name='time_series_inputs')
    
    if CONFIGS['model_type'] == 'flatten':
        flatten = Flatten(name='flatten')(time_series_inputs)
    elif CONFIGS['model_type'] == 'cnn1d':
        conv_0 = Conv1D(16, 3, 2, activation='relu', name='conv_0')(time_series_inputs)
        pool_0 = MaxPool1D(2, name='pool_0')(conv_0)
        conv_1 = Conv1D(32, 3, 2, activation='relu', name='conv_1')(pool_0)
        pool_1 = MaxPool1D(2, name='pool_1')(conv_1)
        flatten = Flatten(name='flatten')(pool_1)
    elif CONFIGS['model_type'] == 'cnn2d':
        reshape = Reshape(target_shape=(
            CONFIGS['window_size'], len(CONFIGS['time_series_cols']), 1
        ), name='reshape')(time_series_inputs)
        conv_0 = Conv2D(8, (3, 1), strides=(2, 1), activation='relu', name='conv_0')(reshape)
        pool_0 = MaxPool2D((2, 1), name='pool_0')(conv_0)
        conv_1 = Conv2D(16, (3, 1), strides=(2, 1), activation='relu', name='conv_1')(pool_0)
        pool_1 = MaxPool2D((2, 1), name='pool_1')(conv_1)
        flatten = Flatten(name='flatten')(pool_1)
    elif CONFIGS['model_type'] == 'lstm':
        lstm_0 = LSTM(16, return_sequences=True, activation='relu', name='lstm_0')(time_series_inputs)
        lstm_1 = LSTM(32, activation='relu', name='lstm_1')(lstm_0)
        flatten = Flatten(name='flatten')(lstm_1)
    elif CONFIGS['model_type'] == 'bilstm':
        bilstm_0 = Bidirectional(LSTM(
            16, return_sequences=True, activation='relu'
        ), name='bilstm_0')(time_series_inputs)
        bilstm_1 = Bidirectional(LSTM(
            32, activation='relu'
        ), name='bilstm_1')(bilstm_0)
        flatten = Flatten(name='flatten')(bilstm_1)
        
    target_date_info_inputs = Input(batch_shape=(
        None, len(CONFIGS['target_date_info_cols'])
    ), name='target_date_info_inputs')
    
    concat = Concatenate(name='concat')([flatten, target_date_info_inputs])
        
    dense_0 = Dense(64, activation='relu', name='dense_0')(concat)
    dense_1 = Dense(32, activation='relu', name='dense_1')(dense_0)
    outputs = Dense(CONFIGS['target_length'], name='outputs')(dense_1)
    
    if not model_name:
        model_name = CONFIGS['model_name']
    
    model = Model(
        inputs = [time_series_inputs, target_date_info_inputs],
        outputs = outputs,
        name = model_name
    )
    
    optimizer = Adam(learning_rate=CONFIGS['learning_rate'])
    model.compile(
        loss = MeanSquaredError(),
        optimizer = optimizer,
        metrics=[inversed_rmse_metric],
    )
    
    if print_summary:
        model.summary()
    
    return model

In [16]:
model = set_model(CONFIGS, print_summary=True)

Model: "multi_input"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
time_series_inputs (InputLayer) [(None, 168, 12)]    0                                            
__________________________________________________________________________________________________
conv_0 (Conv1D)                 (None, 83, 16)       592         time_series_inputs[0][0]         
__________________________________________________________________________________________________
pool_0 (MaxPooling1D)           (None, 41, 16)       0           conv_0[0][0]                     
__________________________________________________________________________________________________
conv_1 (Conv1D)                 (None, 20, 32)       1568        pool_0[0][0]                     
________________________________________________________________________________________

In [17]:
def train_model(model, train_ds, valid_ds, CONFIGS):
    
    early_stop = EarlyStopping(
        patience=CONFIGS['es_patience']
    )
    save_best_only = ModelCheckpoint(
        filepath = f'{CONFIGS["model_path"]}{CONFIGS["model_name"]}.h5',
        monitor = 'val_loss',
        save_best_only = True,
        save_weights_only = True
    )
    
    history = model.fit(
        train_ds,
        batch_size = CONFIGS['batch_size'],
        epochs = CONFIGS['epochs'],
        validation_data = valid_ds,
        callbacks = [
            early_stop,
            save_best_only,
        ]
    )
    
    return history

In [18]:
history = train_model(model, train_ds, valid_ds, CONFIGS)

Epoch 1/100
24/24 [==============================] - 2s 11ms/step - loss: 0.4282 - <lambda>: 81.6231 - val_loss: 1.4411 - val_<lambda>: 150.3956
Epoch 2/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3616 - <lambda>: 74.6825 - val_loss: 1.2644 - val_<lambda>: 140.4767
Epoch 3/100
24/24 [==============================] - 0s 3ms/step - loss: 0.3231 - <lambda>: 70.4842 - val_loss: 1.1809 - val_<lambda>: 135.3461
Epoch 4/100
24/24 [==============================] - 0s 3ms/step - loss: 0.2874 - <lambda>: 66.4756 - val_loss: 1.0873 - val_<lambda>: 129.3643
Epoch 5/100
24/24 [==============================] - 0s 3ms/step - loss: 0.2564 - <lambda>: 62.8235 - val_loss: 0.9897 - val_<lambda>: 122.8575
Epoch 6/100
24/24 [==============================] - 0s 3ms/step - loss: 0.2302 - <lambda>: 59.5877 - val_loss: 0.8975 - val_<lambda>: 116.4187
Epoch 7/100
24/24 [==============================] - 0s 3ms/step - loss: 0.2077 - <lambda>: 56.6472 - val_loss: 0.8109 - val_<lambda>: 

Epoch 58/100
24/24 [==============================] - 0s 3ms/step - loss: 0.0386 - <lambda>: 23.5811 - val_loss: 0.4009 - val_<lambda>: 77.6133
Epoch 59/100
24/24 [==============================] - 0s 3ms/step - loss: 0.0382 - <lambda>: 23.4432 - val_loss: 0.3999 - val_<lambda>: 77.5663


In [19]:
best_model = set_model(CONFIGS, model_name='best_'+CONFIGS['model_name'])
best_model.load_weights(f'{CONFIGS["model_path"]}{CONFIGS["model_name"]}.h5')

In [20]:
y_train_pred = best_model.predict(train_ds)
y_valid_pred = best_model.predict(valid_ds)
y_test_pred = best_model.predict(test_ds)

In [21]:
train_loss, train_rmse = best_model.evaluate(train_ds, verbose=0)
valid_loss, valid_rmse = best_model.evaluate(valid_ds, verbose=0)
test_loss, test_rmse = best_model.evaluate(test_ds, verbose=0)

print(f'train_loss: {train_loss:.6f}\ttrain_rmse: {train_rmse:.6f}')
print(f'valid_loss: {valid_loss:.6f}\tvalid_rmse: {valid_rmse:.6f}')
print(f'test_loss: {test_loss:.6f}\ttest_rmse: {test_rmse:.6f}')

train_loss: 0.042584	train_rmse: 24.965242
valid_loss: 0.399541	valid_rmse: 77.184723
test_loss: 0.308599	test_rmse: 70.677444
